In [4]:
import pandas as pd
import numpy as np

### The following will compute the cosine similarity between two users rating vectors
cosine_similarity = dot_product of two user ratings/(norm(user1 ratging vector)* norm(user2 rating vecor))

In [5]:
def cosine_similarity(r1,r2):
    dot_prod = np.dot(r1,r2)
    #print(dot_prod)
    prod_norms = np.linalg.norm(r1)*np.linalg.norm(r2)
    #print(prod_norms)
    return(dot_prod/prod_norms)

In [9]:

lst = [
                    [5,4,0,0,3,2], 
                    [4,5,3,0,2,3],
                    [3,5,0,0,0,5], 
                    [2,3,5,5,0,0],
                    [3,0,4,3,5,5]   
                  ]

df = pd.DataFrame(lst,columns=['I1','I2','I3','I4','I5','I6'])
# computer the cosine similarity of the first and second rows of the above matrix
print(cosine_similarity(df.iloc[0],df.iloc[1]))
print(cosine_similarity(df.iloc[0],df.iloc[2]))

np.random.randint(low=0, high=5, size=(5,5))

0.8915304664476421
0.7972410051791008


array([[1, 4, 1, 1, 0],
       [2, 1, 0, 2, 3],
       [2, 1, 1, 1, 2],
       [4, 3, 3, 0, 4],
       [0, 4, 3, 2, 0]])

In [7]:
# Initialize the user vs ratings array
# Each row is one user 
# Each column is one item 
# The values are ratings

lst = [
                    [5,4,0,0,3,2], 
                    [4,5,3,0,2,3],
                    [3,5,0,0,0,5], 
                    [2,3,5,5,0,0],
                    [3,0,4,3,5,5]   
                  ]

df = pd.DataFrame(lst,columns=['I1','I2','I3','I4','I5','I6'])
#df = pd.DataFrame(np.random.randn(1000, 500))
#df = pd.DataFrame(np.random.randint(low=0, high=5, size=(1000,500)))

#print(df.sum(axis=1))

df['sum'] = df.sum(axis=1)
print(df)

df['count'] =(df != 0).sum(axis=1)-1
print(df)
# Subtract the mean of each row from all the elements of the rows
# this makes the values to zero mean
df_new = df.apply(lambda x: (x-df['sum'])/df['count'])
print(df_new)
df.drop(['sum','count'],axis=1,inplace=True)
df_new.drop(['sum','count'],axis=1,inplace=True)
print(df_new)
# normalize the column by dividing by the number of values
#df_new = df_new.apply(lambda x: (x/len(df_new.columns)),axis=1)


#print(df_new)

# to retain the zeros in the modified dataframe
df_new = df*df_new
df_new = df_new/df
df_new = df_new.fillna(0)
print(df_new)

   I1  I2  I3  I4  I5  I6  sum
0   5   4   0   0   3   2   14
1   4   5   3   0   2   3   17
2   3   5   0   0   0   5   13
3   2   3   5   5   0   0   15
4   3   0   4   3   5   5   20
   I1  I2  I3  I4  I5  I6  sum  count
0   5   4   0   0   3   2   14      4
1   4   5   3   0   2   3   17      5
2   3   5   0   0   0   5   13      3
3   2   3   5   5   0   0   15      4
4   3   0   4   3   5   5   20      5
         I1        I2        I3        I4        I5        I6  sum     count
0 -2.250000 -2.500000 -3.500000 -3.500000 -2.750000 -3.000000  0.0 -2.500000
1 -2.600000 -2.400000 -2.800000 -3.400000 -3.000000 -2.800000  0.0 -2.400000
2 -3.333333 -2.666667 -4.333333 -4.333333 -4.333333 -2.666667  0.0 -3.333333
3 -3.250000 -3.000000 -2.500000 -2.500000 -3.750000 -3.750000  0.0 -2.750000
4 -3.400000 -4.000000 -3.200000 -3.400000 -3.000000 -3.000000  0.0 -3.000000
         I1        I2        I3        I4        I5        I6
0 -2.250000 -2.500000 -3.500000 -3.500000 -2.750000 -3.000000


In [10]:
# Find centered cosine similarities for U1
list_cosines = [0]
for i in range(1,len(df_new)):
    list_cosines.append(cosine_similarity(df_new.iloc[0],df_new.iloc[i]))
    
cosines = np.array(list_cosines)
cosines

array([0.        , 0.88504622, 0.83415505, 0.49547556, 0.65808107])

In [9]:
(3*0.885+5*0.495+4*0.658) /np.sum(cosines)

2.7019332160146017

In [11]:
np.sum(0.885+0.495+0.658)
#np.sum(cosines)
15/4

3.75

In [12]:
np.dot(df.iloc[:,2] , cosines) /np.sum(cosines)

2.7029220723605074

In [13]:
np.dot(df.iloc[:,3] , cosines) /np.sum(cosines)

1.5495983849685784

### The following computes the centered cosine similary between all combinations of the user rating vectors

In [11]:
# Find centered cosine similarities for all users rating vectors
list_cosines_all = []
for i in range(0,len(df_new)):
    list_cosines = []
    for j in range(0,len(df_new)):
        if i != j:
            list_cosines.append(cosine_similarity(df_new.iloc[i],df_new.iloc[j]))
        else:
            list_cosines.append(0)
    list_cosines_all.append(list_cosines)
    #print(list_cosines)
##list_cosines_all

In [12]:
# predict rating
def predict_rating(col,cosines):
    return(np.dot(df.iloc[:,col] , cosines) /np.sum(cosines))


In [13]:
# now predict the missing ratings
predict_rating(1,list_cosines_all[4])

4.247314176306841

### Finally compute the missing rating values using the weighted cosine similarity of the rating vectors and the ratings
### This uses all the user ratings but this can use first K-most similar user ratings vectors 

In [14]:
# To predict all missing ratings
#
for i in range(0,len(df)):
    for j in range(0,len(df.columns)):
        if(df.iloc[i,j] == 0):
            p=predict_rating(j,list_cosines_all[i])
            print('user {} and item {} rating is {}'.format(i,j,p))

user 0 and item 2 rating is 2.7029220723605074
user 0 and item 3 rating is 1.5495983849685784
user 1 and item 3 rating is 1.8489356452614487
user 2 and item 2 rating is 2.7667795030061373
user 2 and item 3 rating is 1.7765206729154908
user 2 and item 4 rating is 2.405232060257132
user 3 and item 4 rating is 2.4859652473600686
user 3 and item 5 rating is 3.876732117133249
user 4 and item 1 rating is 4.247314176306841


In [266]:
# predit ratings using only the first k closest users
#

Index(['I1', 'I2', 'I3', 'I4', 'I5', 'I6'], dtype='object')

In [288]:
# vowels list
vowels = ['e', 'a', 'u', 'o', 'i']

# sort the vowels
vowels.sort(reverse=True)

# print vowels
print('Sorted list (in Descending):', vowels)

Sorted list (in Descending): ['u', 'o', 'i', 'e', 'a']


In [306]:
(df != 0).sum(axis=1)

0    4
1    5
2    3
3    4
4    5
dtype: int64